In [ ]:
# pymysql을 이용하면 파이썬에서 SQL을 직접 수행할 수 있다

# 다음은 매일 일정한 시간에 네이버 금융 데이터를 스크랩해서 MariaDB에 업데이트하는 DBUpdater 클래스다

# 여기 클래스로 이름붙은 애들 잘 보자, 얘들은 전부 클래스이름별로 각 파일로 빠져나가야할 애들이다
# 지금 작성하는 이 주피터는 그냥 공부용 책으로 생각하면 된다.
# 작성 내용은 책과 동일하게 맞출 것이고, 실제 DB CRUD내용은 커스텀해서 파일로 만들어나가자

import pymysql
import pandas as pd
import requests
from datetime import datetime

class DBUpdater:
    def __init__(self):
        """생성자: MariaDB 연결 및 종목코드 딕셔너리 생성""" # charset utf8로 해주는건 컬럼에 한글 들어갈 때 오류를 피하기 위함
        self.conn = pymysql.connect(host='localhost', port=3306, db='Investar', user='root', passwd='@@@@@@@', charset='utf8')

        with self.conn.cursor() as curs:
            sql = """
            CREATE TABLE IF NOT EXISTS COMPANY_INFO
            (
                CODE VARCHAR(20),
                COMPANY VARCHAR(20),
                LAST_UPDATE DATE,
                PRIMARY KEY (CODE)
            )
            """
            curs.execute(sql)


            sql = """
            CREATE TABLE IF NOT EXISTS DAILY_PRICE
            (
                CODE VARCHAR(20),
                DATE DATE
                OPEN BIGINT(20),
                HIGH BITINT(20),
                LOW BIGINT(20),
                CLOSE BIGINT(20),
                DIFF BIGINT(20),
                VOLUME BIGINT(20),
                PRIMARY KEY (CODE, DATE)
            )
            """
            curs.execute(sql)

        self.conn.commit()

        self.codes = dict()
        self.update_comp_info() # KRX 주식 코드를 읽어와서 COMPANY_INFO 테이블에 업데이트하는 것

    def __del__(self):
        """소멸자: MariaDB 연결 해제"""
        self.conn.close()
    
    def red_krx_code(self):
        """KRX로 부터 상장법인 목록 파일을 읽어와서 DataFrame으로 변환"""

        url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=loadInitPage#'

        # read_html로 읽어들이고
        krx = pd.read_html(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text)[0]

        # 종목코드와 회사명 두개의 컬럼만 남긴다
        # [[ , , , ]] 이런식으로 활용하면 원하는 컬럼을 순서대로 재구성 할 수 있다.
        # ( []하면 Series로 출력되고 [[]] 스타일이 DataFrame으로 출력되는 방식이었던 걸로 기억한다.
        krx = krx[['종목코드', '회사명']]

        # 컬럼이름을 변경하는 함수는 다음과 같다
        krx = krx.rename(columns={'종목코드':'CODE', '회사명':'COMPANY'})

        # 종목코드 앞자리 00~ 들을 패딩해주고 총 6자리로 맞춤
        krx.code = krx.code.map('{:06d}'.format)

        return krx
    
    def update_comp_info(self):
        """종목코드를 company_info 테이블에 업데이트한 후 딕셔너리에 저장"""
        
        sql =  "SELECT * FROM COMPANY_INFO"

        df = pd.read_sql(sql, self.conn) # SQL을 일단 읽음

        for idx in range(len(df)):
            self.codes[df['CODE'].values[idx]] = df['COMPANY'].values[idx] # 위에서 읽은 내용으로 codes라는 딕셔너리에 종목과 회사이름 넣음
        
        with self.conn.cursor() as curs:
            sql = "SELECT MAX(LAST_UPDATE) FROM COMPANY_INFO"
            curs.execute(sql)
            rs = curs.fetchone() # DB에서 가장 최근 업데이트 읽자를 가져온다.
            today = datetime.today().strftime('%Y-%m-%d')

            if rs[0] == None or rs[0].strftime('%Y-%m-%d') < today: # 위에서 구한 마지막 업데이트일자가 없거나, 오늘보다 과거인 경우만 업데이트 한다.
                krx = self.read_krx_code()

                for idx in range(len(krx)):
                    code = krx.code.values[idx]
                    company = krx.company.values[idx]
                    sql = f"""
                    REPLATE INTO COMPANY_INFO
                    (
                        CODE,
                        COMPANY,
                        LAST_UPDATE
                    )
                    VALUES
                    (
                        '{code}',
                        '{company},
                        '{today}
                    )
                    """
                    curs.execute(sql) # REPLACE INTO를 활용해서 종목코드, 회사명, 오늘날짜 행들을 DB에 저장한다

                    self.codes[code] = company # codes 딕셔너리에 키-값 쌍으로 종목코드와 회사명을 추가한다

                    tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')

                    print(f"[{tmnow}] {idx:04d} REPLACE INTO COMPANY_INFO COMPANY "\
                    f"VALUES ({code}, {company}, {today}")
                
                self.conn.commit()
                print(' ')
                    